In [3]:
import numpy as np
import math
from collections import defaultdict

jointCode = {0: 'Head',
             1: 'Neck',
             2: 'Right_Shoulder',
             3: 'Right_Elbow',
             4: 'Right_Wrist',
             5:'Left_Shoulder',
             6:'Left_Elbow',
             7:'Left_Wrist',
             8:'Right_Hip',
             9:'Right_Knee',
             10:'Right_Ankle',
             11:'Left_Hip',
             12:'Left_Knee',
             13:'Left_Ankle',
             14:'Chest',
             15:'Background'}

jointFrameMap = defaultdict(list)

# def adjustSpeed(frames, time): #may not be needed
#     newData = []
#     i = 0
#     l = len(frames)
    
#     a = int(time)
#     b = math.ceil(time)
#     print(a, b)
#     while i < l:
#         newData.append(frames[i])
#         if (i + a <= l) and (i + b) <= l:
#             spliceFrame = ((frames[int(i + a)], frames[int(i + b)]))
#             newData.append(
#                 tuple(map(lambda y: sum(y) / float(len(y)), zip(*spliceFrame))))
            
#         i += int(time * 2)
        
#     return newData


points1 = [(347, 125)]
points2 = [(356, 129)]
points3 = [(378, 137)]
points4 = [(323, 138)]
allPoints = [points1, points2, points3, points4]

def congregate(points):
    for i in range(len(points[0])):
        for j in range(len(points)):
            jointFrameMap[jointCode[i]].append(points[j][i])
            
congregate(allPoints)


In [2]:

codeMap = {
    (1, 0): 0,
    (1, 1): 1,
    (0, 1): 2,
    (-1, 1): 3,
    (-1, 0): 4,
    (-1, -1): 5,
    (0, -1): 6,
    (1, -1): 7
}
# This function generates the chaincode
# for transition between two neighbour points
''' 
dx dy    chain  direction
1  0       0        right
1  1       1        upper-right
0  1       2        up
-1 1       3        upper-left
-1 0       4        left
-1 -1      5        lower-left
0 -1       6        down
1 -1       7        lower-right
'''

def getChainCode(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return codeMap[(dx, dy)]


'''This function generates the list of
chaincodes for given list of points'''


def generateChainCode(linePoints):
    chainCode = []
    for i in range(len(linePoints) - 1):
        a = linePoints[i]
        b = linePoints[i + 1]
        chainCode.append(getChainCode(a[0], a[1], b[0], b[1]))
    return chainCode


def applyBresenham(x1, y1, x2, y2):
    linePoints = []
    linePoints.append([x1, y1])
    xdif = x2 - x1
    ydif = y2 - y1
    dx = abs(xdif)
    dy = abs(ydif)
    if(xdif > 0):
        xs = 1
    else:         
        xs = -1
    if (ydif > 0): 
        ys = 1
    else:          
        ys = -1
    if (dx > dy):

        d = 2 * dy - dx
        while (x1 != x2):
            x1 += xs
            if (d >= 0):
                y1 += ys
                d -= 2 * dx
            d += 2 * dy
            linePoints.append([x1, y1])
    else:

        d = 2 * dx-dy
        while(y1 != y2):
            y1 += ys
            if (d >= 0):
                x1 += xs
                d -= 2 * dy
            d += 2 * dx
            linePoints.append([x1, y1])
    return linePoints
 
 
def test():
    (x1, y1) = (347, 125)
    (x2, y2) = (356, 129)
    linePoints = applyBresenham(x1, y1, x2, y2)
    chainCode = generateChainCode(linePoints)
    chainCodeString = "".join(str(x) for x in chainCode)
    print ('Chain code for the straight line from', (x1, y1),
            'to', (x2, y2), 'is', chainCodeString)
    
test()    

'''
3 2 1 
4   0
5 6 7

'''
#010101010

Chain code for the straight line from (347, 125) to (356, 129) is 010101010


'\n3 2 1 \n4   0\n5 6 7\n\n'

In [ ]:
arr1 = [[(695, 140), (667, 187), (667, 219), (723, 281), (723, 234), (640, 203), (584, 281), (584, 234), (667, 344), (695, 469), (723, 610), (640, 344), (640, 453), (612, 563), (640, 281)],
         [(695, 140), (667, 187), (667, 219), (723, 281), (723, 234), (640, 203), (584, 281), (584, 234), (667, 344), (695, 469), (723, 610), (640, 344), (640, 453), (612, 563), (640, 281)],
         [(723, 140), (695, 187), (695, 219), (723, 281), (751, 219), (640, 203), (584, 281), (640, 234), (667, 344), (695, 469), (723, 610), (640, 344), (640, 453), (612, 563), (667, 281)],
         [(723, 140), (667, 187), (695, 219), (723, 281), (751, 219), (640, 203), (584, 266), (640, 234), (667, 344), (695, 453), (723, 610), (640, 344), (640, 453), (612, 563), (667, 281)],
         [(723, 140), (695, 187), (695, 219), (723, 281), (751, 219), (640, 203), (584, 266), (640, 234), (667, 344), (695, 453), (723, 610), (640, 344), (640, 453), (612, 563), (640, 281)],
         [(723, 140), (667, 203), (695, 219), (723, 281), (751, 234), (640, 219), (584, 281), (640, 234), (640, 344), (695, 453), (723, 610), (612, 344), (612, 453), (612, 563), (667, 281)],
         [(723, 140), (695, 203), (695, 234), (723, 297), (751, 234), (640, 219), (584, 281), (640, 250), (640, 360), (695, 469), (723, 610), (612, 360), (612, 469), (612, 563), (640, 297)],
         [(723, 156), (667, 219), (695, 234), (723, 297), (751, 250), (640, 234), (584, 297), (640, 266), (640, 375), (723, 469), (723, 594), (612, 360), (612, 469), (612, 563), (640, 297)],
         [(723, 187), (695, 234), (695, 266), (723, 328), (751, 266), (640, 250), (584, 313), (612, 281), (640, 391), (723, 485), (723, 610), (612, 375), (612, 469), (612, 563), (640, 328)],
         [(723, 187), (695, 250), (695, 281), (723, 344), (751, 281), (640, 266), (584, 328), (584, 281), (640, 406), (723, 485), (723, 610), (612, 406), (612, 485), (612, 563), (640, 344)],
         [(723, 219), (695, 281), (695, 297), (723, 360), (751, 313), (640, 297), (584, 344), (584, 313), (640, 422), (751, 500), (723, 610), (612, 422), (612, 485), (612, 563), (640, 360)],
         [(723, 234), (695, 297), (695, 313), (723, 391), (751, 328), (640, 313), (584, 360), (640, 328), (640, 438), (751, 500), (723, 610), (612, 438), (751, 500), (612, 563), (640, 375)],
         [(723, 250), (695, 313), (695, 328), (723, 406), (751, 344), (640, 328), (584, 375), (640, 344), (640, 453), (751, 500), (723, 610), (612, 453), (612, 547), (612, 563), (667, 391)],
         [(723, 266), (695, 328), (695, 344), (723, 422), (751, 360), (640, 344), (584, 391), (584, 344), (640, 469), (751, 500), (723, 610), (612, 453), (612, 516), (612, 563), (667, 406)],
         [(723, 281), (667, 344), (695, 360), (723, 422), (751, 375), (640, 344), (584, 406), (612, 375), (640, 469), (751, 500), (723, 610), (612, 469), (612, 532), (612, 563), (667, 406)],
         [(723, 281), (695, 344), (695, 360), (723, 438), (751, 375), (640, 360), (584, 406), (612, 375), (640, 485), (751, 500), (723, 610), (612, 469), (612, 532), (612, 563), (667, 422)],
         [(723, 281), (667, 344), (695, 360), (723, 422), (751, 375), (640, 360), (584, 406), (612, 375), (640, 469), (751, 500), (723, 610), (612, 469), (612, 516), (612, 563), (667, 422)],
         [(723, 266), (695, 313), (695, 344), (723, 406), (751, 360), (640, 328), (584, 391), (612, 360), (640, 453), (751, 500), (723, 610), (612, 453), (640, 516), (612, 563), (667, 391)],
         [(723, 219), (695, 281), (695, 297), (723, 360), (751, 313), (640, 281), (584, 344), (612, 313), (640, 422), (779, 500), (723, 610), (612, 422), (612, 485), (612, 563), (667, 360)],
         [(723, 172), (695, 234), (695, 266), (723, 328), (751, 266), (640, 250), (584, 313), (640, 281), (640, 375), (723, 485), (723, 610), (612, 375), (612, 469), (612, 563), (667, 313)],
         [(695, 140), (695, 203), (695, 234), (723, 297), (751, 234), (640, 219), (584, 281), (640, 250), (640, 360), (695, 469), (723, 610), (612, 360), (612, 453), (723, 610), (667, 297)],
         [(723, 140), (667, 203), (695, 219), (723, 281), (751, 234), (640, 219), (584, 281), (640, 250), (667, 344), (695, 438), (723, 610), (640, 344), (640, 453), (612, 563), (667, 281)],
         [(723, 140), (695, 187), (695, 219), (723, 281), (751, 219), (640, 203), (584, 281), (640, 234), (667, 344), (695, 453), (723, 610), (640, 344), (640, 453), (612, 563), (667, 281)],
         [(695, 125), (667, 187), (695, 219), (723, 281), (751, 219), (640, 203), (584, 281), (640, 234), (667, 344), (695, 453), (723, 610), (640, 344), (640, 453), (612, 563), (667, 281)],
         [(695, 125), (667, 187), (695, 219), (723, 281), (751, 219), (640, 203), (584, 281), (640, 250), (667, 344), (695, 453), (723, 610), (640, 344), (640, 453), (612, 563), (667, 281)],
         [(695, 125), (667, 187), (695, 219), (723, 281), (751, 219), (640, 203), (584, 281), (612, 250), (667, 344), (695, 453), (723, 610), (640, 344), (640, 453), (612, 563), (640, 281)]]

arr2 = [[(695, 140), (667, 187), (667, 219), (723, 281), (723, 234), (640, 203), (584, 281), (584, 234), (667, 344), (695, 469), (723, 610), (640, 344), (640, 453), (612, 563), (640, 281)],
         [(695, 140), (667, 187), (667, 219), (723, 281), (723, 234), (640, 203), (584, 281), (584, 234), (667, 344), (695, 469), (723, 610), (640, 344), (640, 453), (612, 563), (640, 281)],
         [(723, 140), (695, 187), (695, 219), (723, 281), (751, 219), (640, 203), (584, 281), (640, 234), (667, 344), (695, 469), (723, 610), (640, 344), (640, 453), (612, 563), (667, 281)],
         [(723, 140), (667, 187), (695, 219), (723, 281), (751, 219), (640, 203), (584, 266), (640, 234), (667, 344), (695, 453), (723, 610), (640, 344), (640, 453), (612, 563), (667, 281)],
         [(723, 140), (695, 187), (695, 219), (723, 281), (751, 219), (640, 203), (584, 266), (640, 234), (667, 344), (695, 453), (723, 610), (640, 344), (640, 453), (612, 563), (640, 281)],
         [(723, 140), (667, 203), (695, 219), (723, 281), (751, 234), (640, 219), (584, 281), (640, 234), (640, 344), (695, 453), (723, 610), (612, 344), (612, 453), (612, 563), (667, 281)],
         [(723, 140), (695, 203), (695, 234), (723, 297), (751, 234), (640, 219), (584, 281), (640, 250), (640, 360), (695, 469), (723, 610), (612, 360), (612, 469), (612, 563), (640, 297)],
         [(723, 156), (667, 219), (695, 234), (723, 297), (751, 250), (640, 234), (584, 297), (640, 266), (640, 375), (723, 469), (723, 594), (612, 360), (612, 469), (612, 563), (640, 297)],
         [(723, 187), (695, 234), (695, 266), (723, 328), (751, 266), (640, 250), (584, 313), (612, 281), (640, 391), (723, 485), (723, 610), (612, 375), (612, 469), (612, 563), (640, 328)],
         [(723, 187), (695, 250), (695, 281), (723, 344), (751, 281), (640, 266), (584, 328), (584, 281), (640, 406), (723, 485), (723, 610), (612, 406), (612, 485), (612, 563), (640, 344)],
         [(723, 219), (695, 281), (695, 297), (723, 360), (751, 313), (640, 297), (584, 344), (584, 313), (640, 422), (751, 500), (723, 610), (612, 422), (612, 485), (612, 563), (640, 360)],
         [(723, 234), (695, 297), (695, 313), (723, 391), (751, 328), (640, 313), (584, 360), (640, 328), (640, 438), (751, 500), (723, 610), (612, 438), (751, 500), (612, 563), (640, 375)],
         [(723, 250), (695, 313), (695, 328), (723, 406), (751, 344), (640, 328), (584, 375), (640, 344), (640, 453), (751, 500), (723, 610), (612, 453), (612, 547), (612, 563), (667, 391)],
         [(723, 266), (695, 328), (695, 344), (723, 422), (751, 360), (640, 344), (584, 391), (584, 344), (640, 469), (751, 500), (723, 610), (612, 453), (612, 516), (612, 563), (667, 406)],
         [(723, 281), (667, 344), (695, 360), (723, 422), (751, 375), (640, 344), (584, 406), (612, 375), (640, 469), (751, 500), (723, 610), (612, 469), (612, 532), (612, 563), (667, 406)],
         [(723, 281), (695, 344), (695, 360), (723, 438), (751, 375), (640, 360), (584, 406), (612, 375), (640, 485), (751, 500), (723, 610), (612, 469), (612, 532), (612, 563), (667, 422)],
         [(723, 281), (667, 344), (695, 360), (723, 422), (751, 375), (640, 360), (584, 406), (612, 375), (640, 469), (751, 500), (723, 610), (612, 469), (612, 516), (612, 563), (667, 422)],
         [(723, 266), (695, 313), (695, 344), (723, 406), (751, 360), (640, 328), (584, 391), (612, 360), (640, 453), (751, 500), (723, 610), (612, 453), (640, 516), (612, 563), (667, 391)],
         [(723, 219), (695, 281), (695, 297), (723, 360), (751, 313), (640, 281), (584, 344), (612, 313), (640, 422), (779, 500), (723, 610), (612, 422), (612, 485), (612, 563), (667, 360)],
         [(723, 172), (695, 234), (695, 266), (723, 328), (751, 266), (640, 250), (584, 313), (640, 281), (640, 375), (723, 485), (723, 610), (612, 375), (612, 469), (612, 563), (667, 313)],
         [(695, 140), (695, 203), (695, 234), (723, 297), (751, 234), (640, 219), (584, 281), (640, 250), (640, 360), (695, 469), (723, 610), (612, 360), (612, 453), (723, 610), (667, 297)],
         [(723, 140), (667, 203), (695, 219), (723, 281), (751, 234), (640, 219), (584, 281), (640, 250), (667, 344), (695, 438), (723, 610), (640, 344), (640, 453), (612, 563), (667, 281)],
         [(723, 140), (695, 187), (695, 219), (723, 281), (751, 219), (640, 203), (584, 281), (640, 234), (667, 344), (695, 453), (723, 610), (640, 344), (640, 453), (612, 563), (667, 281)],
         [(695, 125), (667, 187), (695, 219), (723, 281), (751, 219), (640, 203), (584, 281), (640, 234), (667, 344), (695, 453), (723, 610), (640, 344), (640, 453), (612, 563), (667, 281)],
         [(695, 125), (667, 187), (695, 219), (723, 281), (751, 219), (640, 203), (584, 281), (640, 250), (667, 344), (695, 453), (723, 610), (640, 344), (640, 453), (612, 563), (667, 281)],
         [(695, 125), (667, 187), (695, 219), (723, 281), (751, 219), (640, 203), (584, 281), (612, 250), (667, 344), (695, 453), (723, 610), (640, 344), (640, 453), (612, 563), (640, 281)]]

In [5]:
from comparator import Comparator
import time
start = time.time()
            
a = Comparator(arr1, arr2)
print(a.score())

end = time.time()
print(end - start)
            

100.0
0.00900125503540039
